### SETTING UP ENVIRONMENT


In [3]:
# flask prerequisites
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!pip install gdal
!pip install tensorflow

#configuring auth tokens
!ngrok authtoken 2P2lZYlo9s5FaHE09zy7l60RMeZ_7qYyzFZrG2tD5E3BmhwLT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15979 sha256=ea2679b1bbe5d9978c79ce7f37d41df61a42e65a66c027611d1b322ee89334af
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [1]:
# mounting google colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
# importing necessary dependencies
# deloyment
from flask import Flask,request, jsonify
from flask.templating import render_template
from flask_ngrok import run_with_ngrok

# deep learning
import keras
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf
import keras.backend as K
from keras.models import load_model
from keras.utils import custom_object_scope


# data manipulation
import pandas as pd

# for mathematical computations
import numpy as np

#for visualization
import matplotlib.pyplot as plt
import seaborn as sns

#GIS
from osgeo import gdal


In [18]:
# Load the trained model
model_path = "/content/gdrive/MyDrive/LULC_deployment/TRAINED_MODEL_5.hdf5"

# Define custom functions
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + K.epsilon())

# Load the model with custom functions
with custom_object_scope({'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m, 'jacard_coef': jacard_coef}):
    model = load_model(model_path)


In [24]:
#defining the template folder
template_folder = "/content/gdrive/MyDrive/LULC_deployment/templates"

# instatiating the flask app
app = Flask(__name__,template_folder=template_folder)
run_with_ngrok(app)


# Set the patch size and stride
patch_size = 256
stride = 128

# Preprocessing function
def data_preprocessing(image):
  sat_image = gdal.Open(image)
  sat_image_array = np.transpose(sat_image.ReadAsArray(), (1, 2, 0))
  # Compute the number of patches in each dimension
  num_patches_x = int(np.ceil((sat_image_array.shape[0] - patch_size) / stride)) + 1
  num_patches_y = int(np.ceil((sat_image_array.shape[1] - patch_size) / stride)) + 1
  num_patches = num_patches_x * num_patches_y
  # Create an array to store the predicted patches
  predicted_patches = np.zeros((num_patches, patch_size, patch_size, 8))
  # Perform predictions on each patch
  for i in range(num_patches_x):
    for j in range(num_patches_y):
      # Compute the coordinates of the patch
      x1 = i * stride
      y1 = j * stride
      x2 = x1 + patch_size
      y2 = y1 + patch_size
      
      # Extract the patch and transpose it to have the channel dimension as the last axis
      patch = sat_image_array[x1:x2, y1:y2, :]
      
      # Perform prediction on the patch
      prediction = model.predict(np.expand_dims(patch, axis=0))

      # Store the predicted patch
      predicted_patches[i*num_patches_y+j, :, :, :] = prediction[0]

  # Reshape predicted patches array into a 3D array with the same shape as the original image
  predicted_image_array = np.zeros((sat_image_array.shape[0], sat_image_array.shape[1], 8))
  count_array = np.zeros((sat_image_array.shape[0], sat_image_array.shape[1]))
  for i in range(num_patches_x):
    for j in range(num_patches_y):
      
      # Compute the coordinates of the patch
      x1 = i * stride
      y1 = j * stride
      x2 = x1 + patch_size
      y2 = y1 + patch_size

      # Add the predicted patch to the predicted image
      predicted_image_array[x1:x2, y1:y2, :] += predicted_patches[i*num_patches_y+j, :, :, :]
      count_array[x1:x2, y1:y2] += 1

  # Compute the mean of the predicted patches where the count is non-zero
  predicted_image_array = predicted_image_array / np.expand_dims(count_array, axis=-1)

  # Convert the predicted image array to an RGB image
  predicted_image_array_rgb = predicted_image_array.argmax(axis=-1)

  return sat_image_array, predicted_image_array_rgb

#definning the home page
@app.route('/')
@app.route('/home')
def Home_page():
    return render_template('home.html')

@app.route('/prediction',  methods=['GET', 'POST'])
def upload_files():
  if request.method == 'POST':
    
    # Get the uploaded files
    uploaded_file1 = request.files['file1']
    uploaded_file2 = request.files['file2']
    
    # Save the uploaded files
    uploaded_file1.save(uploaded_file1.filename)
    uploaded_file2.save(uploaded_file2.filename)

    # Call the preprocessing function on the satellite image
    uploaded_sat_array, predicted_image_array_rgb = data_preprocessing(uploaded_file1.filename)

    # Open the ground truth label using gdal
    gt_label = gdal.Open(uploaded_file2.filename)
    gt_label_array = gt_label.ReadAsArray()
    
    # Return the results as a JSON object
    # Render the template with the processed images
    return render_template('prediction.html',
                            satellite_img=uploaded_file1.filename,
                            predicted_img=predicted_image_array_rgb,
                            gt_img=uploaded_file2.filename)
  else:
        return render_template('prediction.html')
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a9f4-34-125-116-116.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [29/Apr/2023 10:47:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2023 10:47:37] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2023 10:47:47] "GET /prediction HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2023 10:47:47] "GET /static/ HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2023 10:50:23] "POST /prediction HTTP/1.1" 400 -
